In [1]:
from analytics import sheets_api as sheets
from analytics import sheets_elements as elements
from analytics import api as ga
import pandas as pd
from constants import *

In [3]:
ga_authentication, drive_authentication = ga.authenticate(
    SECRET_NAME,
    ga.ga4_service_params,
    ga.drive_service_params,
    port=OAUTH_PORT
)

date_string = f"{START_DATE_CURRENT} - {END_DATE_CURRENT}"

default_params = {
    "service_system": ga_authentication,
    "start_date": START_DATE_CURRENT,
    "end_date": END_DATE_CURRENT,
}

hca_portal_params = {
    **default_params,
    "base_dimension_filter": HCA_BROWSER_EXCLUDE_FILTER,
    "property": HCA_PORTAL_ID,
}



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=425030666072-vun85q7nt3038skng8gs0f03juh97e17.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8082%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=gQXcCGWIVhfxgIYwz935kc6SSgk4Wp&access_type=offline


In [4]:
df_pageviews = elements.get_page_views_change(hca_portal_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR) 

In [5]:
dict_spreadsheet = {
    "Page Views": df_pageviews,
}
sheets.fill_spreadsheet_with_df_dict(
    sheets.create_sheet_in_folder(
        drive_authentication,
        SHEET_NAME,
        PARENT_FOLDER_NAME,
        override_behavior=sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE
    ),
    dict_spreadsheet,
    sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE,
    column_formatting_options={
        "Outbound Links": {
            "Total Clicks Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            "Total Users Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },
        "Page Views": {
            "Total Views Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            "Total Users Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        }

    }
)